# Dialogue Summarization Task using generative AI

## 1 - Set up Kernel and Required Dependencies

In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 47.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 require

In [2]:
# Load the datasets, Large Language Model (LLM), tokenizer, and configurator

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## 2 - Summarize Dialogue without Prompt Engineering

Let's upload some simple dialogues from the DialogSum Hugging Face dataset. This dataset contains 10,000+ dialogues with the corresponding manually labeled summaries and topics.

In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

# Print a couple of dialogues with their baseline summaries

indices = [33, 150]
line = '-'.join('' for x in range(100))

for i, index in enumerate(indices):
    print(line)
    print('Example ', i + 1)
    print(line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(line)
    print()

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: Welcome to my birthday party, I am so happy you can come.
#Person2#: Thanks for inviting me. Here is the gift for you. Happy birthday, Francis! Many more happy and healthy years for you!
#Person1#: Thank you, shall I open it now?
#Person2#: Yes, please do.
#Person1#: Wow, a remote car model and my favorite brand. I really like it. That is so nice of you.
#Person2#: Yeah, I was really struggling whether I should give you this nice little car. It was the last one they had and I really like it so much myself.
#Person1#: Typical you, always wanting to keep the best things for yourself. The more I appreciate the gift now.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person2# com

Load the FLAN-T5 model, creating an instance of the AutoModelForSeq2SeqLM class with the .from_pretrained() method.

In [4]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [5]:
# To perform encoding and decoding, you need to work with text in a tokenized form

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Test the tokenizer encoding and decoding a simple sentence:

sent = "Welcome to my birthday party, I am so happy you can come."
sent_enc = tokenizer(sent, return_tensors='pt')
sent_dec= tokenizer.decode(
        sent_enc["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sent_enc["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sent_dec)



ENCODED SENTENCE:
tensor([5242,   12,   82, 3591, 1088,    6,   27,  183,   78, 1095,   25,   54,
         369,    5,    1])

DECODED SENTENCE:
Welcome to my birthday party, I am so happy you can come.




Now it's time to explore how well the base LLM summarizes a dialogue without any prompt engineering. Prompt engineering is an act of a human changing the prompt (input) to improve the response for a given task.


In [6]:
for i, index in enumerate(indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(line)
    print('Example ', i + 1)
    print(line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: Welcome to my birthday party, I am so happy you can come.
#Person2#: Thanks for inviting me. Here is the gift for you. Happy birthday, Francis! Many more happy and healthy years for you!
#Person1#: Thank you, shall I open it now?
#Person2#: Yes, please do.
#Person1#: Wow, a remote car model and my favorite brand. I really like it. That is so nice of you.
#Person2#: Yeah, I was really struggling whether I should give you this nice little car. It was the last one they had and I really like it so much myself.
#Person1#: Typical you, always wanting to keep the best things for yourself. The more I appreciate the gift now.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person2# comes

## 3 - Summarize Dialogue with an Instruction Prompt

Prompt engineering is an important concept in using foundation models for text generation.


### 3.1 - Zero Shot Inference with an Instruction Prompt

In order to instruct the model to perform a task - summarize a dialogue - you can take the dialogue and convert it into an instruction prompt. This is often called zero shot inference.

Wrap the dialogue in a descriptive instruction and see how the generated text will change:

In [7]:


for i, index in enumerate(indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(line)
    print('Example ', i + 1)
    print(line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')



---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Welcome to my birthday party, I am so happy you can come.
#Person2#: Thanks for inviting me. Here is the gift for you. Happy birthday, Francis! Many more happy and healthy years for you!
#Person1#: Thank you, shall I open it now?
#Person2#: Yes, please do.
#Person1#: Wow, a remote car model and my favorite brand. I really like it. That is so nice of you.
#Person2#: Yeah, I was really struggling whether I should give you this nice little car. It was the last one they had and I really like it so much myself.
#Person1#: Typical you, always wanting to keep the best things for yourself. The more I appreciate the gift now.

Summary:
    
------------------------------------------------------------------------------------

### 3.2 - Zero Shot Inference with the Prompt Template from FLAN-T5

Let's use a slightly different prompt. FLAN-T5 has many prompt templates that are published for certain tasks here. In the following code, you will use one of the pre-built FLAN-T5 prompts:


In [8]:
for i, index in enumerate(indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(line)
    print('Example ', i + 1)
    print(line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

#Person1#: Welcome to my birthday party, I am so happy you can come.
#Person2#: Thanks for inviting me. Here is the gift for you. Happy birthday, Francis! Many more happy and healthy years for you!
#Person1#: Thank you, shall I open it now?
#Person2#: Yes, please do.
#Person1#: Wow, a remote car model and my favorite brand. I really like it. That is so nice of you.
#Person2#: Yeah, I was really struggling whether I should give you this nice little car. It was the last one they had and I really like it so much myself.
#Person1#: Typical you, always wanting to keep the best things for yourself. The more I appreciate the gift now.

What was going on?

---------------------------------------------------------------------------------------------------
BASELI

## 4 - Summarize Dialogue with One Shot and Few Shot Inference

One shot and few shot inference are the practices of providing an LLM with either one or more full examples of prompt-response pairs that match your task - before your actual prompt that you want completed. This is called "in-context learning" and puts your model into a state that understands your specific task. You can read more about it in this blog from HuggingFace.



### 4.1 - One Shot Inference

Let's build a function that takes a list of example_indices_full, generates a prompt with full examples, then at the end appends the prompt which you want the model to complete (example_index_to_summarize). You will use the same FLAN-T5 prompt template from section 3.2.

In [9]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""

    return prompt

In [10]:
# Construct the prompt to perform one shot inference:

example_indices_full = [33]
example_index_to_summarize = 150

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)
print(one_shot_prompt)


Dialogue:

#Person1#: Welcome to my birthday party, I am so happy you can come.
#Person2#: Thanks for inviting me. Here is the gift for you. Happy birthday, Francis! Many more happy and healthy years for you!
#Person1#: Thank you, shall I open it now?
#Person2#: Yes, please do.
#Person1#: Wow, a remote car model and my favorite brand. I really like it. That is so nice of you.
#Person2#: Yeah, I was really struggling whether I should give you this nice little car. It was the last one they had and I really like it so much myself.
#Person1#: Typical you, always wanting to keep the best things for yourself. The more I appreciate the gift now.

What was going on?
#Person2# comes to the birthday party of Francis and sends him a remote car model.



Dialogue:

#Person1#: Taxi!
#Person2#: Where will you go, sir?
#Person1#: Friendship Hotel.
#Person2#: OK, it's not far from here.
#Person1#: I have something important to do, can you fast the speed?
#Person2#: Sure, I'll try my best. Here we are

In [11]:
# pass this prompt to perform the one shot inference:

summary = dataset['test'][example_index_to_summarize]['summary']
inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')



---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# takes a taxi to the Friendship Hotel for something important.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
The taxi driver will pick up Person1 at Friendship Hotel at 20 yuan.


### 4.2 - Few Shot Inference

Let's explore few shot inference by adding two more full dialogue-summary pairs to your prompt.


In [12]:
example_indices_full = [33, 50, 100]
example_index_to_summarize = 150
few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)
print(few_shot_prompt)


Dialogue:

#Person1#: Welcome to my birthday party, I am so happy you can come.
#Person2#: Thanks for inviting me. Here is the gift for you. Happy birthday, Francis! Many more happy and healthy years for you!
#Person1#: Thank you, shall I open it now?
#Person2#: Yes, please do.
#Person1#: Wow, a remote car model and my favorite brand. I really like it. That is so nice of you.
#Person2#: Yeah, I was really struggling whether I should give you this nice little car. It was the last one they had and I really like it so much myself.
#Person1#: Typical you, always wanting to keep the best things for yourself. The more I appreciate the gift now.

What was going on?
#Person2# comes to the birthday party of Francis and sends him a remote car model.



Dialogue:

#Person1#: Yeah. Just pull on this strip. Then peel off the back.
#Person2#: You might make a few enemies this way.
#Person1#: If they don't think this is fun, they're not meant to be our friends.
#Person2#: You mean your friends. I th

In [13]:
# pass this prompt to perform a few shot inference:

summary = dataset['test'][example_index_to_summarize]['summary']
inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')



Token indices sequence length is longer than the specified maximum sequence length for this model (845 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# takes a taxi to the Friendship Hotel for something important.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
The taxi driver will take Person1 to Friendship Hotel at 20 yuan.
